# How the App Works

## 1. User Interaction
- The user enters transaction and financial information into the input fields.
- Selects a country from the dropdown.
- Clicks the **"Predict Sales"** button.

---

## 2. Backend Request
- Sends the input data as JSON to the FastAPI service (`http://localhost:8000/predict`).
- Receives the predicted sales value from the response.

---

## 3. Result Display
- Shows the prediction as a formatted metric (e.g., **£1234.56**).
- Provides a **bar chart** summarizing the key inputs for clarity.
- Displays **raw input data** for review.

---

## 4. Error Handling
- If the backend is offline or the prediction fails:
  - Displays **error messages** to inform the user.
  - Prevents users from accessing broken workflows.


In [2]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import plotly.express as px
import json

# Configure page settings
st.set_page_config(
    page_title="Retail Sales Prediction",
    page_icon="🛍️",
    layout="wide"
)

def create_prediction_ui():
    """Create the main prediction UI"""
    
    # Title and description
    st.title("🛍️ Retail Sales Prediction")
    st.markdown("""
    This application predicts retail sales based on various features. 
    Enter the required information below to get a prediction.
    """)
    
    # Create two columns for input fields
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Transaction Information")
        num_purchases = st.number_input(
            "Number of Purchases",
            min_value=1,
            value=10,
            help="Total number of purchases made"
        )
        
        total_quantity = st.number_input(
            "Total Quantity",
            min_value=1,
            value=100,
            help="Total quantity of items purchased"
        )
        
        avg_quantity = st.number_input(
            "Average Quantity per Transaction",
            min_value=0.1,
            value=10.0,
            help="Average quantity of items per transaction"
        )
        
        std_quantity = st.number_input(
            "Standard Deviation of Quantity",
            min_value=0.1,
            value=2.0,
            help="Variation in quantity across transactions"
        )
    
    with col2:
        st.subheader("Financial Information")
        avg_transaction = st.number_input(
            "Average Transaction Amount",
            min_value=0.1,
            value=50.0,
            help="Average amount per transaction"
        )
        
        std_transaction = st.number_input(
            "Standard Deviation of Transaction Amount",
            min_value=0.1,
            value=10.0,
            help="Variation in transaction amounts"
        )
        
        avg_unit_price = st.number_input(
            "Average Unit Price",
            min_value=0.1,
            value=5.0,
            help="Average price per unit"
        )
        
        std_unit_price = st.number_input(
            "Standard Deviation of Unit Price",
            min_value=0.1,
            value=1.0,
            help="Variation in unit prices"
        )
    
    # Country selection
    st.subheader("Location Information")
    country = st.selectbox(
        "Select Country",
        options=[
            "United Kingdom", "Germany", "France", "EIRE", "Spain",
            "Netherlands", "Belgium", "Switzerland", "Portugal", "Australia"
        ]
    )
    
    # Create prediction button
    if st.button("Predict Sales", type="primary"):
        # Prepare input data
        input_data = {
            "num_purchases": num_purchases,
            "total_quantity": total_quantity,
            "avg_quantity": avg_quantity,
            "std_quantity": std_quantity,
            "avg_transaction": avg_transaction,
            "std_transaction": std_transaction,
            "avg_unit_price": avg_unit_price,
            "std_unit_price": std_unit_price,
            "country": country
        }
        
        try:
            # Make prediction request to FastAPI
            response = requests.post(
                "http://localhost:8000/predict",
                json=input_data
            )
            
            if response.status_code == 200:
                result = response.json()
                
                # Create results container
                st.success("Prediction successful!")
                
                # Display results in a nice format
                col1, col2 = st.columns(2)
                
                with col1:
                    st.metric(
                        label="Predicted Sales",
                        value=f"£{result['prediction']:,.2f}"
                    )
                
                # Create a summary dataframe
                summary_df = pd.DataFrame({
                    'Metric': ['Total Quantity', 'Average Transaction', 'Average Unit Price'],
                    'Value': [total_quantity, avg_transaction, avg_unit_price]
                })
                
                # Plot summary metrics
                with col2:
                    fig = px.bar(
                        summary_df,
                        x='Metric',
                        y='Value',
                        title='Input Summary'
                    )
                    st.plotly_chart(fig)
                
                # Display detailed input data
                with st.expander("View Detailed Input Data"):
                    st.json(input_data)
                
            else:
                st.error(f"Error making prediction: {response.text}")
                
        except requests.exceptions.ConnectionError:
            st.error("Error: Could not connect to the prediction service. Please make sure the FastAPI service is running.")
        except Exception as e:
            st.error(f"An error occurred: {str(e)}")

def check_api_health():
    """Check if the FastAPI service is running"""
    try:
        response = requests.get("http://localhost:8000/health")
        return response.status_code == 200
    except:
        return False

def main():
    # Add sidebar
    st.sidebar.title("About")
    st.sidebar.info(
        """
        This application provides sales predictions for retail data using 
        a machine learning model. Enter your data in the input fields and 
        click 'Predict Sales' to get a prediction.
        """
    )
    
    # Check API health
    api_status = check_api_health()
    status_container = st.sidebar.container()
    
    if api_status:
        status_container.success("✅ API Service: Online")
    else:
        status_container.error("❌ API Service: Offline")
        st.error("The prediction service is currently offline. Please start the FastAPI service.")
        return
    
    # Create the main UI
    create_prediction_ui()
    
    # Add footer
    st.markdown("""---""")
    st.markdown(
        """
        <div style='text-align: center'>
            <small>Retail Sales Prediction App • Created with Streamlit</small>
        </div>
        """,
        unsafe_allow_html=True
    )

if __name__ == "__main__":
    main()


Writing app.py


In [3]:
!streamlit run app.py

^C
